#Thresholding

In this episode, we will learn how to use skimage functions to apply thresholding to an image. Thresholding is a type of *image segmentation*, where we change the pixels of an image to make the image easier to analyze. In thresholding, we convert an image from color or grayscale into a *binary image*, i.e., one that is simply black and white. Most frequently, we use thresholding as a way to select areas of interest of an image, while ignoring the parts we are not concerned with. We have already done some simple thresholding, in the "Manipulating pixels" section of the Skimage Images episode. In that case, we used a simple NumPy array manipulation to separate the pixels belonging to the root system of a plant from the black background. In this episode, we will learn how to use skimage functions to perform thresholding. Then, we will use the masks returned by these functions to select the parts of an image we are interested in.

##Simple thresholding

Consider this image, with a series of crudely cut shapes set against a white background. The black outline around the image is not part of the image.

<img src="https://datacarpentry.org/image-processing/fig/06-junk-before.jpg" alt="paper shapes" style="float: left; margin-right:10px;"/>

Now suppose we want to select only the shapes from the image. In other words, we want to leave the pixels belonging to the shapes "on," while turning the rest of the pixels "off," by setting their color channel values to zeros. The `skimage` library has several different methods of thresholding. We will start with the simplest version, which involves an important step of human input. Specifically, in this simple, *fixed-level thresholding*, we have to provide a threshold value, `t`.

The process works like this. First, we will load the original image, convert it to grayscale, and blur it with one of the methods from the Blurring episode. Then, we will use the `>` operator to apply the threshold `t`, a number in the closed range $\left[0.0, 1.0\right]$. Pixels with color values on one side of `t` will be turned "on," while pixels with color values on the other side will be turned "off." In order to use this function, we have to determine a good value for `t`. How might we do that? Well, one way is to look at a grayscale histogram of the image. Here is the histogram produced by the code from the Creating Histograms episode, if we run it on the colored shapes image shown above.

<img src="https://datacarpentry.org/image-processing/fig/06-junk-histogram.png" alt="paper shapes histogram" style="float: left; margin-right:10px;"/>

Since the image has a white background, most of the pixels in the image are white. This corresponds nicely to what we see in the histogram: there is a spike near the value of 1.0. If we want to select the shapes and not the background, we want to turn off the white background pixels, while leaving the pixels for the shapes turned on. So, we should choose a value of `t` somewhere before the large peak and turn pixels above that value "off".

Here are the first few lines of Python code to apply simple thresholding to the image, to accomplish this task.

In [ ]:
# one-time imports and configuration
import numpy as np
import skimage.color
import skimage.filters
import skimage.io

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

from matplotlib import pyplot as plt

In [ ]:
# values for blurring and threshold
k = 2
t = 0.8

# read and display original image
image = skimage.io.imread('https://i.imgur.com/c1Y4NyB.jpg')
skimage.io.imshow(image)
plt.show()

In [ ]:
# grayscale and blur before thresholding
blur = skimage.color.rgb2gray(image)
blur = skimage.filters.gaussian(blur, sigma=k)

The fixed-level thresholding is performed using `numpy` comparison operators.

Here, we want to turn "on" all pixels which have values smaller than the threshold, so we use the less-than operator `<` to compare the blurred image, `blur`, to the threshold `t`. The operator returns a binary image, that we capture in the variable `mask`. It has only one channel, and each of its values is either 0 or 1. 

After executing the code cell below, you will see the binary image created by the thresholding operation. The program used parameters of `sigma = 2` and `t = 0.8` to produce this image. You can see that the areas where the shapes were in the original area are now white, while the rest of the mask image is black.

In [ ]:
# perform fixed thresholding
mask = blur < t

# display the mask
skimage.io.imshow(mask)
plt.show()

We can now apply the mask to the original colored image as we learned in the Drawing and Bitwise Operations episode. We should be left with only the colored shapes from the original.

In [ ]:
# use the mask to select the "interesting" parts of the image
sel = np.zeros_like(image)
sel[mask] = image[mask]

# display the result
skimage.io.imshow(sel)
plt.show()

---
> **Practice with simple thresholding**
> 
> Suppose we want to use simple thresholding to select only the colored 
> shapes from this image, found at <a href="https://i.imgur.com/dPgHUFQ.jpg">https://i.imgur.com/dPgHUFQ.jpg</a>:
> 
> <img src="https://datacarpentry.org/image-processing/fig/06-more-junk.jpg" alt="more shapes" style="float: left; margin-right:10px;"/>
> 
> First, use techniques from the histogram lesson (Lesson D) 
> to examine the grayscale histogram of the image. Determine a good 
> candidate for the threshold value `t`.
> 
> Now, use code like that shown above to turn the pixels *above* the 
> threshold value `t` off. Then, using the resulting mask, produce an
> image with only the colored shapes on a pure black background. There
> is no need to display all of the intermediate images; only show the
> final result.
---

In [ ]:
# TODO: write your code here



##Adaptive thresholding

There are also `skimage` methods to perform *adaptive thresholding*. The chief advantage of adaptive thresholding is that the value of the threshold, `t`, is determined automatically for us. One such method, *Otsu's method*, is particularly useful for situations where the grayscale histogram of an image has two peaks. Consider this maize root system image, which we have seen before in a previous lesson:

<img src="https://datacarpentry.org/image-processing/fig/06-roots-original.jpg" alt="maize roots" style="float: left; margin-right:10px;"/>

Now, look at the grayscale histogram of this image, as produced by our histogram code from the Creating Histograms episode.

<img src="https://datacarpentry.org/image-processing/fig/06-roots-histogram.png" alt="maize root histogram" style="float: left; margin-right:10px;"/>

The histogram has a significant peak around 0.2, and a second, albeit smaller peak very near 1.0. Thus, this image is a good candidate for thresholding with Otsu’s method. The mathematical details of how this work are complicated (see the <a href="https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_thresholding.html">`skimage` documentation</a> if you are interested), but the outcome is that Otsu's method finds a threshold value between the two peaks of a grayscale histogram.

The `skimage.filters.threshold_otsu()` function can be used to determine the adaptive threshold via Otsu's method. Then numpy comparison operators can be used to apply it as before.

To start off demonstrating Otsu's method, we first read and display our target image.

In [ ]:
# sigma blur value
k = 1.0

# read and display original image
image = skimage.io.imread('https://i.imgur.com/y2Mxtzy.jpg')
skimage.io.imshow(image)
plt.show()

In [ ]:
# grayscale and blur
blur = skimage.color.rgb2gray(image)
blur = skimage.filters.gaussian(blur, sigma=k)

# determine threshold value
t = skimage.filters.threshold_otsu(blur)

# create mask
mask = blur > t

The function `skimage.filters.threshold_otsu()` uses Otsu's method to automatically determine the threshold value based on its inputs' grayscale histogram and returns it. Then, we use the comparison operator `>` for binary thesholding. As we have seen before, pixels above the threshold value will be turned on, those below the threshold will be turned off.

For this root image, and a Gaussian blur with a sigma of 1.0, the computed threshold value is 0.42, and we can see the mask by displaying it in the usual manner.

In [ ]:
skimage.io.imshow(mask)
plt.show()

Next, we can use it to select only the "interesting" parts of the image and display the results.

In [ ]:
# mask the image
sel = np.zeros_like(image)
sel[mask] = image[mask]

# display the result
skimage.io.imshow(sel)
plt.show()

##Application: measuring root mass

Let us now turn to an application where we can apply thresholding and other techniques we have learned to this point. Consider these four maize root system images.

<img src="https://datacarpentry.org/image-processing/fig/07-four-maize-roots.jpg" alt="maize roots collage" style="float: left; margin-right:10px;"/>

Now suppose we are interested in the amount of plant material in each image, and in particular how that amount changes from image to image. Perhaps the images represent the growth of the plant over time, or perhaps the images show four different maize varieties at the same phase of their growth. In any case, the question we would like to answer is, "how much root mass is in each image?" We will construct Python code to measure this value for a single image, and then create loop to execute the code on each trial image in turn.

Our strategy will be this:

1. Read the image, converting it to grayscale as it is read. For this application we do not need the color image.
2. Blur the image.
3. Use Otsu's method of thresholding to create a binary image, where the pixels that were part of the maize plant are white, and everything else is black.
4. Save the binary image so it can be examined later.
5. Count the white pixels in the binary image, and divide by the number of pixels in the image. This ratio will be a measure of the root mass of the plant in the image.
5. Output the name of the image processed and the root mass ratio.

The cells below contain Python code to implement this root-mass-measuring strategy. Almost all of the code should be familiar, and in fact, it may seem simpler than the code we have worked on thus far, because we are not displaying any of the images with this program. Our code here is intended to run and produce its numeric result -- a measure of the root mass in the image -- without human intervention.

The four images are available online via these links:

Trial 16: <a href="https://i.imgur.com/0awactO.jpg">https://i.imgur.com/0awactO.jpg</a>

Trial 20: <a href="https://i.imgur.com/7VPYzgp.jpg">https://i.imgur.com/7VPYzgp.jpg</a>

Trial 216: <a href="https://i.imgur.com/vvOBNvk.jpg">https://i.imgur.com/vvOBNvk.jpg</a>

Trial 293: <a href="https://i.imgur.com/aW4Z39M.jpg">https://i.imgur.com/aW4Z39M.jpg</a>

The names for the images, and their links, are placed in a dictionary in the first cell.


In [ ]:
imageInfo = {'trial-016':'https://i.imgur.com/0awactO.jpg', 
             'trial-020':'https://i.imgur.com/7VPYzgp.jpg',
             'trial-216':'https://i.imgur.com/vvOBNvk.jpg',
             'trial-293':'https://i.imgur.com/aW4Z39M.jpg'}

The next cell shows how to read one of the images, by fetching its URL out of the dictionary, based on the trial name. We read the image as grayscale immediately, rather than reading as color and then converting to grayscale.

In [ ]:
# read the original image, converting to grayscale
imgName = 'trial-016'
image = skimage.io.imread(imageInfo[imgName], as_gray=True)

Next the grayscale image is blurred with a Gaussian that is defined by the sigma parameter, contained in the variable `k`.

In [ ]:
# blur before thresholding
k = 1.5

blur = skimage.filters.gaussian(image, sigma=k)

Following that, we create a binary image with Otsu's method for thresholding, just as we did in the previous section. Since the program is intended to produce numeric output, without a person shepherding it, it does not display any of the images.

In [ ]:
# perform adaptive thresholding to produce a binary image
t = skimage.filters.threshold_otsu(blur)
binary = blur > t

We do, however, want to save the binary images, in case we wish to examine them at a later time. This cell does so by appending a `-binary.png` suffix to the name of the image, which we stored previously in the `imgName` variable. The binary image is saved via a call to the `skimage.io.imsave()` function. In order to convert from the binary range of 0 and 1 of the mask to a gray level image that can be saved as `.png`, we use the `skimage.img_as_ubyte()` utility function.

In [ ]:
# save binary image
binaryFileName = imgName + '-binary.png'
skimage.io.imsave(binaryFileName, skimage.img_as_ubyte(binary))

Finally, we can examine the code that is the reason this program exists! This block of code determines the root mass ratio in the image.

In [ ]:
# determine root mass ratio
rootPixels = np.count_nonzero(binary)
w = binary.shape[1]
h = binary.shape[0]
density = rootPixels / (w * h)

# output in format suitable for .csv
print(imgName, density, sep=",")

Recall that we are working with a binary image at this point; every pixel in the image is either zero (black) or 1 (white). We want to count the number of white pixels, which is easily accomplished with a call to the `np.count_nonzero()` function. Then we determine the width and height of the image, via the first and second elements of the image's shape. Then the density ratio is calculated by dividing the number of white pixels by the total number of pixels in the image. Finally, the code prints out the name of the file processed and the corresponding root density.

We have four images to process in this example, and in a real-world scientific situation, there might be dozens, hundreds, or even thousands of images to process. To save us the tedium of manually running the preceding Python code on each image, we can construct a Python loop to run the code multiple times for us. The following cell does that, assuming that the image names and file names / URLs are held in a Python dictionary. 

We also place the rootmass calculating code in a function, to make the loop code easer to write and read.

In [ ]:
def calculateRootMass(name, url, k):
  '''
  Calculate and print an estimate of the rootmass in a maize root system image.

  Parameters
  ----------
  name : string with a name for the image, e.g., 'trial-016'
  url : string with local file name or URL for the file to examine
  k : sigma value for blur kernel 
  '''
  # load image
  image = skimage.io.imread(url, as_gray=True)

  # blur the image
  blur = skimage.filters.gaussian(image, sigma=k)

  # perform adaptive thresholding to produce a binary image
  t = skimage.filters.threshold_otsu(blur)
  binary = blur > t

  # save binary image
  binaryFileName = name + '-binary.png'
  skimage.io.imsave(binaryFileName, skimage.img_as_ubyte(binary))

  # determine root mass ratio
  rootPixels = np.count_nonzero(binary)
  w = binary.shape[1]
  h = binary.shape[0]
  density = rootPixels / (w * h)

  # output in format suitable for .csv
  print(name, density, sep=",")

In [ ]:
# perform calculations on all of our images
for name in imageInfo:
  calculateRootMass(name, imageInfo[name], 1.5)

These values could be saved in a `.csv` file for later use, either by copying and pasting the above output into a file manually, or by modifying the `calculateRootMass()` function to write to a file instead of printing output to the screen. 

---
> **Ignoring more of the images -- brainstorming**
>
> Let us take a closer look at the binary images produced by the proceeding program.
> <img src="https://datacarpentry.org/image-processing/fig/07-four-maize-roots-binary.jpg" alt="binary maize roots collage" style="float: left; margin-right:10px;"/>
> 
> Our root mass ratios include white pixels that are
> not part of the plant in the image, do they not? 
> The numbered labels and the white circles in each 
> image are preserved during the thresholding, and 
> therefore their pixels are included in our 
> calculations. Those extra pixels might have a 
> slight impact on our root mass ratios, especially 
> the labels, since the labels are not the same size 
> in each image. How might we remove the labels and 
> circles before calculating the ratio, so that our 
> results are more accurate? Brainstorm and think 
> about some options, given what we have learned so 
> far.
---

---
> **Ignoring more of the images -- implementation**
> 
> Using the preceding code as a basis, perform two
> thresholding steps in order to remove the label and
> white circle from each image. First, use simple 
> inverse binary thresholding to remove the label and
> circle, then proceed with Otsu's method as before. 
> 
> For your convenience, the `calculateRootMass()` 
> function is reproduced in the cell below, with 
> comments showing  where to make your modifications.
---

In [ ]:
def calculateRootMass(name, url, k):
  '''
  Calculate and print an estimate of the rootmass in a maize root system image.

  Parameters
  ----------
  name : string with a name for the image, e.g., 'trial-016'
  url : string with local file name or URL for the file to examine
  k : sigma value for blur kernel 
  '''
  # load image
  image = skimage.io.imread(url, as_gray=True)

  # blur the image
  blur = skimage.filters.gaussian(image, sigma=k)

  # TODO: perform binary inverse thresholding to create a mask
  # that selects the white circle and label, so we can remove
  # them later

  # TODO: use the mask you just created to remove the circle
  # and label from the blur image

  # perform adaptive thresholding to produce a binary image
  t = skimage.filters.threshold_otsu(blur)
  binary = blur > t

  # save binary image
  binaryFileName = name + '-binary.png'
  skimage.io.imsave(binaryFileName, skimage.img_as_ubyte(binary))

  # determine root mass ratio
  rootPixels = np.count_nonzero(binary)
  w = binary.shape[1]
  h = binary.shape[0]
  density = rootPixels / (w * h)

  # output in format suitable for .csv
  print(name, density, sep=",")

In [ ]:
# perform calculations on all of our images
for name in imageInfo:
  calculateRootMass(name, imageInfo[name], 1.5)

---
> **Thresholding a bacteria colony image**
> 
> Refer back to the grayscale histograms of the 
> three sample bacteria colony images you created in
> Lesson D. Determine a plausible threshold value
> for each image. Then, write code to threshold a
> grayscale version of each image, leavining the 
> pixels in the bacterial colonies "on," while 
> turning the rest of the pixels in the image "off."
>
> The URLs for the colony images are:
>
> Colony image 1: <a href='https://i.imgur.com/uM0Rt9r.png'>https://i.imgur.com/uM0Rt9r.png</a>
> 
> Colony image 2: <a href='https://i.imgur.com/MAWoq9A.png'>https://i.imgur.com/MAWoq9A.png</a>
> 
> Colony image 3: <a href='https://i.imgur.com/SrG8kTQ.png'>https://i.imgur.com/SrG8kTQ.png</a>
---

In [ ]:
# TODO: write thresholding code for the colony images here
